In [12]:
import pandas as pd
from pprint import pprint
from IPython.display import Image
from sklearn.model_selection import train_test_split

## Reading Data

In [13]:
# GALAXY DATASET
def DatasetGALAXY():
    df = pd.read_csv('../../Data/galaxymorphology/dataset1_sydney.csv')
    df.head()
    df['target'].value_counts()
    mapping = {
        'elliptical': 0,
        'spiral':1
    }
    df['target'] = df['target'].map(mapping)
    df = df.sample(df.shape[0])
    return df
df = DatasetGALAXY()

In [25]:
#CAR DATASET
def DatasetCAR():
    df = pd.read_csv('../../Data/dataset1-car/car_dataset.csv')
    
#     df['buying'].replace(['vhigh', 'high', 'med', 'low'],[3,2,1,0], inplace = True)
#     df['maintenance'].replace(['vhigh', 'high', 'med', 'low'],[3,2,1,0], inplace = True)
#     df['doors'].replace(['2', '3', '4', '5more'],[2,3,4,5], inplace = True)
#     df['persons'].replace(['2', '4', 'more'],[2,4,5], inplace = True)
#     df['lug_boot'].replace(['small', 'med', 'big'],[1,2,3], inplace = True)
#     df['safety'].replace(['high', 'med', 'low'],[3,2,1], inplace = True)
#     df['target'].replace(['unacc', 'acc', 'good', 'vgood'],[0,1,2,3], inplace = True)
    df= df[df.target !=3]
    df= df[df.target !=2]
    df = df.sample(df.shape[0])
    return df

df = DatasetCAR()


In [36]:
#IRIS DATASET
def DatasetIRIS():
    df = pd.read_csv('../../Data/iris/iris.data')
    #df['target'].replace(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],[0,1,2], inplace = True)
    df = df.sample(df.shape[0])
    return df
df = DatasetIRIS()


In [40]:
#TITANIC DATASET
def DatasetTITANIC():
    df = pd.read_csv('../../Data/Titanic/train.csv')
    df.drop(columns = ['Cabin', 'Embarked', 'Ticket','Name','PassengerId'], inplace=True)
    df['Sex'].replace(['male', 'female'],[0,1], inplace = True)
    df =df.dropna()
    
    return df
df = DatasetTITANIC()

In [41]:
df

,Pclass,Sex,Age,SibSp,Parch,Fare,target
0,3,0,22.0,1,0,7.2500,0
1,1,1,38.0,1,0,71.2833,1
2,3,1,26.0,0,0,7.9250,1
3,1,1,35.0,1,0,53.1000,1
4,3,0,35.0,0,0,8.0500,0
...,...,...,...,...,...,...,...
1300,3,1,3.0,1,1,13.7750,0
1302,1,1,37.0,1,0,90.0000,1
1303,3,1,28.0,0,0,7.7750,0
1305,1,1,39.0,0,0,108.9000,1


## NAIVE BAYES

In [28]:
def create_table(df, label_column):
    table = {}

    # determine values for the label
    value_counts = df[label_column].value_counts().sort_index()
    table["class_names"] = value_counts.index.to_numpy()
    table["class_counts"] = value_counts.values
    
    # determine probabilities for the features
    for feature in df.drop(label_column, axis=1).columns:
        table[feature] = {}
        # determine counts
        counts = df.groupby(label_column)[feature].value_counts()
        df_counts = counts.unstack(label_column)
        # add one count to avoid "problem of rare values"
        if df_counts.isna().any(axis=None):
            df_counts.fillna(value=0, inplace=True)
            df_counts += 1

        # calculate probabilities
        df_probabilities = df_counts / df_counts.sum()
        for value in df_probabilities.index:
            probabilities = df_probabilities.loc[value].to_numpy()
            table[feature][value] = probabilities
    pprint(table) 
    return table

In [22]:
def predict_example(row, lookup_table):
    class_estimates = lookup_table["class_counts"]
    for feature in row.index:
        try:
            value = row[feature]
            probabilities = lookup_table[feature][value]
            class_estimates = class_estimates * probabilities

        # skip in case "value" only occurs in test set but not in train set
        # (i.e. "value" is not in "lookup_table")
        except KeyError:
            continue

    index_max_class = class_estimates.argmax()
    prediction = lookup_table["class_names"][index_max_class]
    
    return prediction

In [23]:
def get_overall_cm(df):
    classes = df['target'].nunique()
    if classes < 2:
        classes = 2
    overall_cm = []
    for i in range(classes):
        overall_cm.append( [0] * classes)
    return overall_cm

In [31]:
from sklearn.metrics import confusion_matrix

def Kfold(df,n_folds, test_size, overall_cm):
    # Iterate through all folds
    
    for i in range(n_folds):
        print("\n\n\n\nTHIS IS FOLD:", i)
        #split test and train
        test = df.iloc[test_size * i : test_size*i+test_size]
        test_labels = test.iloc[:, -1]
        test.drop(columns = ['target'], inplace= True)

        train = df.iloc[test_size:]
        lookup_table = create_table(train, label_column="target")

        predictions = test.apply(predict_example, axis=1, args=(lookup_table,))

        # generate and append cm 
        overall_cm = overall_cm + confusion_matrix(test_labels, predictions)
    return overall_cm


# Run Naive Bayes

In [42]:
if __name__ == "__main__":
    # KFOLD take in n folds and the size of dataset
    n_folds = 2
    test_size = int(df.shape[0]/n_folds)

    # get overall cm returns a Confusion matrix with 0's shape depends on (classes)
    nbcm = Kfold(df,n_folds, test_size, get_overall_cm(df))
    print(f"\n\n\n Confusion Matrix\n{nbcm}")





THIS IS FOLD: 0
{'Age': {0.17: array([0.0025    , 0.00668896]),
         0.33: array([0.005     , 0.00334448]),
         0.42: array([0.0025    , 0.00668896]),
         0.67: array([0.0025    , 0.00668896]),
         0.75: array([0.005     , 0.00334448]),
         0.83: array([0.0025    , 0.01003344]),
         0.92: array([0.0025    , 0.00668896]),
         1.0: array([0.005     , 0.01672241]),
         2.0: array([0.0075    , 0.00668896]),
         3.0: array([0.005     , 0.00334448]),
         4.0: array([0.005     , 0.01337793]),
         5.0: array([0.005     , 0.00668896]),
         6.0: array([0.01      , 0.01337793]),
         7.0: array([0.0025    , 0.00668896]),
         8.0: array([0.005     , 0.01003344]),
         9.0: array([0.005     , 0.01003344]),
         10.0: array([0.01      , 0.00334448]),
         11.0: array([0.005     , 0.00668896]),
         11.5: array([0.005     , 0.00334448]),
         12.0: array([0.0025    , 0.01003344]),
         13.0: array([0.0075 

D:\DeveloperTools\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(






THIS IS FOLD: 1
{'Age': {0.17: array([0.0025    , 0.00668896]),
         0.33: array([0.005     , 0.00334448]),
         0.42: array([0.0025    , 0.00668896]),
         0.67: array([0.0025    , 0.00668896]),
         0.75: array([0.005     , 0.00334448]),
         0.83: array([0.0025    , 0.01003344]),
         0.92: array([0.0025    , 0.00668896]),
         1.0: array([0.005     , 0.01672241]),
         2.0: array([0.0075    , 0.00668896]),
         3.0: array([0.005     , 0.00334448]),
         4.0: array([0.005     , 0.01337793]),
         5.0: array([0.005     , 0.00668896]),
         6.0: array([0.01      , 0.01337793]),
         7.0: array([0.0025    , 0.00668896]),
         8.0: array([0.005     , 0.01003344]),
         9.0: array([0.005     , 0.01003344]),
         10.0: array([0.01      , 0.00334448]),
         11.0: array([0.005     , 0.00668896]),
         11.5: array([0.005     , 0.00334448]),
         12.0: array([0.0025    , 0.01003344]),
         13.0: array([0.0075 




 Confusion Matrix
[[524  93]
 [129 298]]


In [ ]:
# predictions = df_test.apply(predict_example, axis=1, args=(lookup_table,))
# predictions.head()


u-g           2.174410
g-r           1.135680
r-i           0.491790
i-z           0.363360
ecc           0.824696
pConc_u       0.574688
pConc_r       0.314069
pConc_z       0.335882
m4_u          0.786833
m4_g          2.381084
m4_r          2.402476
m4_i          2.387196
m4_z          2.372986
petroR50_u    4.401828
petroR50_r    2.454687
petroR50_z    2.012387
petroR90_u    7.659512
petroR90_r    7.815753
petroR90_z    5.991359
Name: 19, dtype: float64
u-g            1.449210
g-r            0.684760
r-i            0.353870
i-z            0.269440
ecc            0.707266
pConc_u        0.571356
pConc_r        0.448379
pConc_z        0.396563
m4_u           2.213931
m4_g           2.387288
m4_r           2.553287
m4_i           2.574248
m4_z           2.599853
petroR50_u     9.076157
petroR50_r     8.857353
petroR50_z     8.754080
petroR90_u    15.885290
petroR90_r    19.754170
petroR90_z    22.074900
Name: 327, dtype: float64
u-g            1.682020
g-r            0.737370
r-i     

19     spiral
327    spiral
466    spiral
336    spiral
460    spiral
dtype: object

In [ ]:
# predictions_correct = predictions == test_labels
# accuracy = predictions_correct.mean()
# print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.500
